In [3]:
import requests
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sea
import matplotlib.colors as matcol
from dotenv import load_dotenv
from pathlib import Path

In [4]:
load_dotenv()
DATE_BEGIN = os.getenv('DATE_BEGIN')
DATE_END = os.getenv('DATE_END')
API_URL = os.getenv('API_URL')

In [5]:
visit_json = requests.get(f"{API_URL}/visits?begin={DATE_BEGIN}&end={DATE_END}")
data_vis=visit_json.json()
df_visits=pd.DataFrame(data_vis)
print(df_visits)

                             datetime platform  \
0       Wed, 01 Mar 2023 10:36:22 GMT      web   
1       Wed, 01 Mar 2023 06:25:00 GMT      web   
2       Wed, 01 Mar 2023 10:26:13 GMT      web   
3       Wed, 01 Mar 2023 12:33:06 GMT      web   
4       Wed, 01 Mar 2023 01:38:35 GMT      web   
...                               ...      ...   
263454  Tue, 29 Aug 2023 10:43:29 GMT      web   
263455  Tue, 29 Aug 2023 19:29:16 GMT      web   
263456  Tue, 29 Aug 2023 12:51:11 GMT      web   
263457  Tue, 29 Aug 2023 21:45:08 GMT      web   
263458  Wed, 30 Aug 2023 19:19:04 GMT      web   

                                               user_agent  \
0       Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...   
1       Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7...   
2       Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...   
3       Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...   
4       Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...   
...                              

In [6]:
registration_json = requests.get(f"{API_URL}/registrations?begin={DATE_BEGIN}&end={DATE_END}")
data_reg=registration_json.json()
df_registrations=pd.DataFrame(data_reg)
print(df_registrations)

                            datetime                       email platform  \
0      Wed, 01 Mar 2023 07:40:13 GMT           ebyrd@example.org      web   
1      Wed, 01 Mar 2023 13:14:00 GMT    knightgerald@example.org      web   
2      Wed, 01 Mar 2023 03:05:50 GMT  cherylthompson@example.com      web   
3      Wed, 01 Mar 2023 00:04:47 GMT       halldavid@example.org      web   
4      Wed, 01 Mar 2023 18:31:52 GMT        denise86@example.net      web   
...                              ...                         ...      ...   
21831  Thu, 31 Aug 2023 07:32:08 GMT         ikelley@example.net      ios   
21832  Thu, 31 Aug 2023 23:25:50 GMT     brittanycox@example.net      ios   
21833  Thu, 31 Aug 2023 19:29:18 GMT           aryan@example.net      ios   
21834  Thu, 31 Aug 2023 19:38:54 GMT       emartinez@example.net      ios   
21835  Thu, 31 Aug 2023 08:36:05 GMT      terriperry@example.net      ios   

      registration_type                               user_id  
0          

In [23]:
df_visits = df_visits[~df_visits['user_agent'].str.contains('bot')]
df_visits['date_group'] = pd.to_datetime(df_visits['datetime']).dt.date
df_visits = df_visits.sort_values(by=['visit_id', 'date_group']).drop_duplicates(subset=['visit_id'], keep='last')
df_visits_grouped = df_visits.groupby(['date_group', 'platform']).size().reset_index(name='visits')
df_visits_grouped

,date_group,platform,visits
0,2023-03-01,android,75
1,2023-03-01,ios,22
2,2023-03-01,web,279
3,2023-03-02,android,67
4,2023-03-02,ios,31
...,...,...,...
547,2023-08-30,ios,66
548,2023-08-30,web,1227
549,2023-08-31,android,57
550,2023-08-31,ios,50


In [27]:
df_registrations['date_group'] = pd.to_datetime(df_registrations['datetime']).dt.date
df_registrations_grouped = df_registrations.groupby(['date_group', 'platform']).size().reset_index(name='registrations')
df_registrations_grouped

,date_group,platform,registrations
0,2023-03-01,android,61
1,2023-03-01,ios,18
2,2023-03-01,web,8
3,2023-03-02,android,59
4,2023-03-02,ios,24
...,...,...,...
547,2023-08-30,ios,40
548,2023-08-30,web,34
549,2023-08-31,android,42
550,2023-08-31,ios,36


In [35]:
df_merged = pd.merge(df_visits_grouped, df_registrations_grouped, on=['date_group', 'platform'], how='left')
df_merged['conversion'] = df_merged['registrations'] / df_merged['visits'] * 100
df_merged

,date_group,platform,visits,registrations,conversion
0,2023-03-01,android,75,61,81.333333
1,2023-03-01,ios,22,18,81.818182
2,2023-03-01,web,279,8,2.867384
3,2023-03-02,android,67,59,88.059701
4,2023-03-02,ios,31,24,77.419355
...,...,...,...,...,...
547,2023-08-30,ios,66,40,60.606061
548,2023-08-30,web,1227,34,2.770986
549,2023-08-31,android,57,42,73.684211
550,2023-08-31,ios,50,36,72.000000


In [39]:
df_merged.to_json('./conversion.json')

In [43]:
ads_date = pd.read_csv('ads.csv')
ads_date['date_ymd'] = pd.to_datetime(ads_date['date']).dt.date
ads_grouped = ads_date.groupby(['date_ymd', 'utm_campaign']).agg({'cost':'sum'}).reset_index()
ads_merged = df_merged.merge(ads_grouped, left_on = 'date_group', right_on = 'date_ymd', how = 'left') 
ads_merged = ads_merged[['date_group', 'visits', 'registrations', 'cost', 'utm_campaign']]
ads_merged['cost'] = ads_merged['cost'].fillna(0)
ads_merged['utm_campaign'] = ads_merged['utm_campaign'].fillna('none')
ads_merged

,date_group,visits,registrations,cost,utm_campaign
0,2023-03-01,75,61,212.0,advanced_algorithms_series
1,2023-03-01,22,18,212.0,advanced_algorithms_series
2,2023-03-01,279,8,212.0,advanced_algorithms_series
3,2023-03-02,67,59,252.0,advanced_algorithms_series
4,2023-03-02,31,24,252.0,advanced_algorithms_series
...,...,...,...,...,...
547,2023-08-30,66,40,109.0,intro_to_python_course
548,2023-08-30,1227,34,109.0,intro_to_python_course
549,2023-08-31,57,42,215.0,intro_to_python_course
550,2023-08-31,50,36,215.0,intro_to_python_course


In [46]:
ads_merged.to_json('./ads.json')

In [48]:
new_folder_path = 'charts'
os.makedirs(new_folder_path, exist_ok=True)